This is the BERT2 quantum hybrid. The BERT layers of the pretrained model are frozen and only the VQC layer is trained (fine-tuned).

Python 3.8, Ubuntu and CUDA 12 are required. A clean Anaconda/Conda environment is essential. Refer to setup instructions and QDNABERT2_working_env.yaml file.


Pennylane dependencies: https://docs.pennylane.ai/en/stable/development/guide/installation.html

Transfer example: https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning

A working BERT implementation can be found here (one line needs to be updated to make this work:
q_out_elem = torch.hstack(quantum_net(elem, self.q_params)).float().unsqueeze(0)

Github link to the repo: https://github.com/sikfeng/quantum-transfer-learning-with-bert

The goal is to run this model on the GUE dataset which must be in the same directory as the finetuning folder (for details refer to classical DNABERT2 setup in this repo).

I have also uploaded a yml file, so you can see my Anaconda environment. My machine has a GForce RTX3070 GPU and Ubuntu is the OS

In [ ]:
#!conda install -c "nvidia/label/cuda-12.2.2" cuda-toolkit

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
! pip install triton==2.0.0.dev20221202
! pip install torch==1.13.1
! pip install einops==0.6.1
! pip install peft==0.4.0
! pip install huggingface-hub==0.16.4
! pip install scikit-learn
! pip install matplotlib
! pip install progressbar

In [ ]:
! pip install tensorboard==2.13.0
! pip install tensorboard-data-server==0.7.1

In [4]:
!pip install pennylane --upgrade
import pennylane as qml

In [5]:
from pennylane import numpy as np
qml.about()

Name: PennyLane
Version: 0.32.0
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /home/beri/anaconda3/envs/QDNABERT2env/lib/python3.8/site-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml, typing-extensions
Required-by: PennyLane-Lightning

Platform info:           Linux-6.2.0-35-generic-x86_64-with-glibc2.17
Python version:          3.8.18
Numpy version:           1.23.5
Scipy version:           1.10.1
Installed devices:
- default.gaussian (PennyLane-0.32.0)
- default.mixed (PennyLane-0.32.0)
- default.qubit (PennyLane-0.32.0)
- default.qubit.autograd (PennyLane-0.32.0)
- default.qubit.jax (PennyLane-0.32.0)
- default.qubit.tf (PennyLane-0.32.0)
- default.qubit.torch (PennyLane-0.32.0)
- default.qutrit (PennyLane-0.32.0)
- null.qubit (PennyLane-0

In [6]:
### SET DIRECTORY
import os
os.chdir("/home/beri/anaconda3/envs/QDNABERT2env") # my directory
print(os.getcwd())

/home/beri/anaconda3/envs/QDNABERT2env


In [ ]:
!git lfs install
!git clone https://huggingface.co/zhihan1996/DNABERT-2-117M

In [7]:
### LOAD PYTHON MODULES
# Load basic modules
import os
import sys
import time
from os import path

# Load data and machine learning modules
import torch
import triton
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset, DataLoader

# Print triton version
print(triton.__version__)

2.0.0


In [7]:
### PRINT GPU DEVICE
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3070


In [8]:
### LOAD DNABERT MODULE
# In the github I already uploaded this: https://github.com/Zhihan1996/DNABERT_2
# Note: I modified the file DNABERT_2/finetune/train.py to solve some bugs.

sys.path.append("/home/beri/anaconda3/envs/QDNABERT2env/finetune/") 
from train import *

In [9]:
### PARAMETERS
model_args=ModelArguments()
data_args=DataArguments()
training_args=TrainingArguments

# better to save the pretrained model "DNABERT-2-117M" somewhere locally
model_args.model_name_or_path="/home/beri/anaconda3/envs/QDNABERT2env/DNABERT-2-117M/"

batchsize=16 # reduce it to decrease CUDA memory

training_args.deepspeed_plugin=None
#training_args.log_level="info"
training_args.run_name="DNABERT2_aug"
training_args.model_max_length=20
training_args.per_device_train_batch_size=batchsize
training_args.per_device_eval_batch_size=batchsize
training_args.gradient_accumulation_steps=5 # increase it to reduce CUDA memory 
training_args.learning_rate=3e-5
training_args.num_train_epochs=4
training_args.fp16=False
training_args.save_steps=0 #400
training_args.evaluation_strategy="steps"
training_args.eval_steps=500 # avoid testing on validation while training too frequently (takes a lot of memory)
training_args.warmup_steps=50
training_args.logging_steps=100000
training_args.find_unused_parameters=False

# Other arguments to add since it was bugging
training_args.device=torch.device('cuda:0')
training_args.report_to=["tensorboard"]
training_args.world_size=1
training_args.per_device_train_batch_size=8
training_args.train_batch_size=batchsize
training_args.eval_batch_size=batchsize
training_args.test_batch_size=batchsize
training_args.batch_size=batchsize
training_args.num_training_steps=200
training_args.n_gpu=1
training_args.distributed_state=None
training_args.local_rank=-1


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
n_qubits = 8 # Number of qubits
q_depth = 12 # Depth of the quantum circuit (number of variational layers)
q_delta = 0.01 # Initial spread of random quantum weights
from torch import nn

In [12]:
dev = qml.device("default.qubit", wires=n_qubits)                                 # default.qubit

In [13]:
# this is the default 4 qubit variational circuit described by the Pennylane team 

def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)


def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT.
    """
    # In other words it should apply something like :
    # CNOT  CNOT  CNOT  CNOT...  CNOT
    #   CNOT  CNOT  CNOT...  CNOT
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])


@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    RY_layer(q_input_features)

    # Sequence of trainable variational layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)


class DressedQuantumNet(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()

        self.n_qubits = n_qubits # Number of qubits
        self.q_depth = q_depth # Depth of the quantum circuit (number of variational layers)
        self.q_delta = q_delta # Initial spread of random quantum weights
        self.pre_net = nn.Linear(768, self.n_qubits)
        self.q_params = nn.Parameter(self.q_delta * torch.randn(self.q_depth * self.n_qubits))
        self.post_net = nn.Linear(self.n_qubits, 2)

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """

        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 768 to 4
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = torch.hstack(quantum_net(elem, self.q_params)).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)

In [14]:
# MAKE CUSTOM MODEL TO MODIFY DNABERT2

import torch.nn as nn
from transformers.modeling_outputs import TokenClassifierOutput

class CustomModel(nn.Module):
  def __init__(self,num_labels): 
    super(CustomModel,self).__init__() 
    self.num_labels = num_labels 

    #Load Model with given checkpoint and extract its body
    self.tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, trust_remote_code=True)
    self.model = AutoModel.from_pretrained(model_args.model_name_or_path, 
                                           trust_remote_code=True, output_hidden_states=True).cuda()
    self.dropout = nn.Dropout(0.1) 

    
    # QUANTUM LAYER 
    self.classifier = DressedQuantumNet() 

  def forward(self, input_ids=None, attention_mask=None,labels=None):
    #Extract outputs from the body
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

    #Add custom layers
    sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
    print(sequence_output.shape)
    
    # By default, no pooling is done, only the first word is taken (sequence_output[:,0,:]). 
    # The authors of BERT paper found it sufficient to use only the output from the 1st token 
    # for few tasks such as classification
    logits = self.classifier(sequence_output[:,0,:].view(-1,768)) # calculate losses
    
    # POOLING FOR QUANTUM TEAM
    #sequence_output_max=torch.max(sequence_output, dim=1) # here global max pooling
    #logits = self.classifier(sequence_output_max)
    
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    
    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs[0])

modeltest=CustomModel(2).cuda()
print(modeltest)

total_params = sum(p.numel() for p in modeltest.parameters())
print(str(total_params)+" parameters")


Some weights of BertModel were not initialized from the model checkpoint at /home/beri/anaconda3/envs/QDNABERT2env/DNABERT-2-117M/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CustomModel(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768, out_features=61

In [15]:
# please check the below is correct - BERT parameters will be frozen and only quantum layer will be trained
# although the model is working there could still be errors here in terms of incorrect class inheritance, 
# so please check this code here very carefully
# you are also free to rewrite the code into a cleaner and perhaps more effective way

model = CustomModel(nn.Module)

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

for name, param in model.named_parameters():
    print(name)

# Use CUDA or CPU according to the "device" object.
model = model.to(device)

Some weights of BertModel were not initialized from the model checkpoint at /home/beri/anaconda3/envs/QDNABERT2env/DNABERT-2-117M/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.embeddings.word_embeddings.weight
model.embeddings.token_type_embeddings.weight
model.embeddings.LayerNorm.weight
model.embeddings.LayerNorm.bias
model.encoder.layer.0.attention.self.Wqkv.weight
model.encoder.layer.0.attention.self.Wqkv.bias
model.encoder.layer.0.attention.output.dense.weight
model.encoder.layer.0.attention.output.dense.bias
model.encoder.layer.0.attention.output.LayerNorm.weight
model.encoder.layer.0.attention.output.LayerNorm.bias
model.encoder.layer.0.mlp.gated_layers.weight
model.encoder.layer.0.mlp.wo.weight
model.encoder.layer.0.mlp.wo.bias
model.encoder.layer.0.mlp.layernorm.weight
model.encoder.layer.0.mlp.layernorm.bias
model.encoder.layer.1.attention.self.Wqkv.weight
model.encoder.layer.1.attention.self.Wqkv.bias
model.encoder.layer.1.attention.output.dense.weight
model.encoder.layer.1.attention.output.dense.bias
model.encoder.layer.1.attention.output.LayerNorm.weight
model.encoder.layer.1.attention.output.LayerNorm.bias
model.encoder.layer.1.mlp.gated_

In [16]:
# PICK GUE DATA CLASS
GUE_class="tf" # pick among: "EMP" "mouse" "prom" "splice" "tf" "virus"

if GUE_class=="EMP":
    GUE_subclasses = ["H3","H3K14ac","H3K36me3","H3K4me1","H3K4me2","H3K4me3","H3K79me3","H3K9ac","H4","H4ac"]

if GUE_class=="mouse":
    GUE_subclasses = ["0", "1", "2", "3", "4"]

if GUE_class=="prom":
    GUE_subclasses = ["prom_300_all","prom_300_notata","prom_300_tata",
                      "prom_core_all","prom_core_notata","prom_core_tata"]

if GUE_class=="splice":
    GUE_subclasses = ["reconstructed"]

if GUE_class=="tf":
    GUE_subclasses = ["0", "1", "2", "3", "4"]

if GUE_class=="virus":
    GUE_subclasses = ["covid"]


In [17]:
from sklearn import metrics

In [18]:
### FINE TUNE DNABERT2


for GUE_subclass in GUE_subclasses:
    
    #GUE_subclass="H3K4me1"

    data_args.data_path="/home/beri/anaconda3/envs/QDNABERT2env/GUE/"+GUE_class+"/"+GUE_subclass
    training_args.output_dir="results/DNABERT2/"+GUE_class+"/"+GUE_subclass

    # load tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=True,
        trust_remote_code=True,
    )

    if "InstaDeepAI" in model_args.model_name_or_path:
        tokenizer.eos_token = tokenizer.pad_token

    # define datasets and data collator
    train_dataset = SupervisedDataset(tokenizer=tokenizer, 
                                      data_path=os.path.join(data_args.data_path, "train.csv"), 
                                      kmer=data_args.kmer)
    val_dataset = SupervisedDataset(tokenizer=tokenizer, 
                                     data_path=os.path.join(data_args.data_path, "dev.csv"), 
                                     kmer=data_args.kmer)
    test_dataset = SupervisedDataset(tokenizer=tokenizer, 
                                     data_path=os.path.join(data_args.data_path, "test.csv"), 
                                     kmer=data_args.kmer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

    
    # load model
    # model used by defaults (no model customization)
    if False:
        model=transformers.AutoModelForSequenceClassification.from_pretrained(
            model_args.model_name_or_path,
            cache_dir=training_args.cache_dir,
            num_labels=train_dataset.num_labels,
            trust_remote_code=True,
            output_hidden_states=False,
        )
    # customised model
    if True:
        model=CustomModel(num_labels=2).cuda()

    # configure LoRA
    if model_args.use_lora:
        lora_config = LoraConfig(
            r=model_args.lora_r,
            lora_alpha=model_args.lora_alpha,
            target_modules=list(model_args.lora_target_modules.split(",")),
            lora_dropout=model_args.lora_dropout,
            bias="none",
            task_type="SEQ_CLS",
            inference_mode=False,
        )
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    # define trainer
    trainer = transformers.Trainer(model=model,
                                   tokenizer=tokenizer,
                                   args=training_args,
                                   compute_metrics=compute_metrics,
                                   train_dataset=train_dataset,
                                   eval_dataset=val_dataset,
                                   data_collator=data_collator)
    trainer.local_rank=training_args.local_rank
    trainer.train()

    if training_args.save_model:
        trainer.save_state()
        safe_save_model_for_hf_trainer(trainer=trainer, output_dir=training_args.output_dir)
        
    
    ### TEST ACCURACY ON INDEPENDENT TEST DATA

    # get the evaluation results from trainer
    if training_args.eval_and_save_results:
        results_path = os.path.join(training_args.output_dir, "metrics")
        results = trainer.evaluate(eval_dataset=test_dataset)
        os.makedirs(results_path, exist_ok=True)
        with open(os.path.join(results_path, "test_results.json"), "w") as f:
            json.dump(results, f)
        
    ### ONLY IF YOU WANT TO SAVE THE FINE-TUNED MODEL (BY DEFAULT: NOT SAVING)
    if False:
        path_model="/home/beri/anaconda3/envs/QDNABERT2env/pytorch_model_finetuned.bin"
        torch.save(model.state_dict(), path_model)
    
    del tokenizer, train_dataset, val_dataset, test_dataset, data_collator, model, trainer
     

Some weights of BertModel were not initialized from the model checkpoint at /home/beri/anaconda3/envs/QDNABERT2env/DNABERT-2-117M/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Currently training with a batch size of: 16
***** Running training *****
  Num examples = 32,378
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Training with DataParallel so batch size has been adjusted to: 16
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 5
  Total optimization steps = 1,616
  Number of trainable parameters = 117,074,810


torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])


Step,Training Loss,Validation Loss,Accuracy,F1,Matthews Correlation,Precision,Recall
500,No log,0.478302,0.788000,0.787992,0.578511,0.789394,0.789116
1000,No log,0.496359,0.778000,0.777928,0.560077,0.780565,0.779513
1500,No log,0.529933,0.763000,0.762475,0.535060,0.769593,0.765483


torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([

torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([

...

torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([

torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([



Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


torch.Size([16, 20, 768])


torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([16, 20, 768])
torch.Size([